<a href="https://colab.research.google.com/github/MrEminent42/wordle-ai/blob/main/WordleAI_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## WordleGame

In [ ]:
from enum import IntEnum
from termcolor import colored


class WordleGame:
    def __init__(self, answer):
        self.answer = answer
        self.board = []
        self.is_over = False
        self.win = False

    def __repr__(self):
        s = ""
        # in each line
        for i, line in enumerate(self.board):
            colors = self.get_colors(self.get_line_string(i))
            for i, tile in enumerate(line):
                # for i, char in enumerate(line):
                s += (
                    colored(tile.char, "white")
                    if tile.color == Color.GREY
                    else (
                        colored(tile.char, "magenta")
                        if tile.color == Color.YELLOW
                        else colored(tile.char, "green")
                    )
                )
            s += "\n"

        return s

    def guess(self, guess):
        """Takes a five-letter guess, records this guess on the game's board.
        Returns the array of Colors with each index corresponding to the color of the letter at that index in the guess"""
        tiles = []
        if len(guess) != 5:
            raise ValueError(
                'Wordle guess must be a 5-letter word. Could not guess with word "'
                + guess
                + '".'
            )
        # convert everything to upper case
        guess = guess.upper()
        colors = self.get_colors(guess)
        # log guess to board
        tiles = [Tile(guess[i], colors[i]) for i in range(5)]
        self.board.append(tiles)

        print("LOGGED COLOR " + str(int(colors[1])))

        # check for game over
        if len(self.board) >= 6:
            self.is_over = True
        elif guess == self.answer:
            self.is_over = self.win = True

        # give back list of colors
        return colors

    def get_colors(self, guess):
        """Takes in a five-letter guess, returns an array of Colors with
        each index corresponding to the color of the letter at that index in the guess."""
        if len(guess) != 5:
            raise ValueError(
                'Can only find colors for words of length 5. Could not find colors for word "'
                + guess
                + '"'
            )
        colors = []
        # occurrences_left = {char: self.answer.count(char) for char in self.answer}
        occurrences_left = {}
        # more efficient way of counting num occurences
        for char in self.answer:
            if char in occurrences_left:
                occurrences_left[char] += 1
            else:
                occurrences_left[char] = 1

        for i, char in enumerate(guess):
            # if the character is in the correct place, green
            if self.answer[i] == char:
                colors.append(Color.GREEN)
                occurrences_left[char] -= 1
            # if the character is in the word, but in the wrong place
            elif char in self.answer:
                # if there are stil occurences of this char that have not been accounted for
                if occurrences_left[char] > 0:
                    # append a yellow tile
                    colors.append(Color.YELLOW)
                    # record that we have accounted for this occurence
                    occurrences_left[char] -= 1
                else:
                    colors.append(Color.GREY)
            else:
                colors.append(Color.GREY)

        return colors

    def get_line_string(self, i):
        s = ""
        for tile in self.board[i]:
            s += tile.char
        return s

    def is_over(self):
        return self.is_over

    def run_game(self):
        print("Welcome to Wordle-AI!")
        while not self.is_over:
            self.guess(input("Guess: "))
            print(self)
        if self.win:
            print("Congrats! You found the word in " + str(len(self.board)) + " tries.")
        else:
            print("Darn! You didn't find the word. It was " + self.answer + ".")


class Color(IntEnum):
    GREY = GRAY = 0
    YELLOW = 1
    GREEN = 2


class Tile:
    def __init__(self, character, color):
        self.char = character
        self.color = color

## DQN
Based on https://www.tensorflow.org/agents/tutorials/1_dqn_tutorial

### TF Setup

In [ ]:
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install pyvirtualdisplay
!pip install tf-agents[reverb]
!pip install pyglet

In [ ]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import reverb

import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.utils import common

### Gym setup

In [ ]:
!pip install "gym>=0.21.0"
!pip install tf-agents

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import numpy as np

import random

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

#### Wordle Data Setup

In [ ]:
import requests
import io

import urllib.request
import os
from os import path

In [ ]:
raw_data_source = "https://raw.githubusercontent.com/tabatkins/wordle-list/main/words"
directory_name = "WordleData"
file_name = "wordList.txt"


def retrieve_data():
    url_request = urllib.request.urlopen(raw_data_source)
    data = url_request.read().decode("utf-8")
    return data


def store_data():
    if os.path.isfile(get_file_path()) == False:
        data = retrieve_data()
        os.mkdir(directory_name)
        file = open(get_file_path(), "w")
        file.write(data)
        file.close


def get_file_path():
    return path.join(directory_name, file_name)


In [ ]:
store_data()

find_word = {}
find_num = {}

with open(get_file_path()) as f:
  words = f.readlines()

for i in range(len(words)):
  find_word[i] = words[i]
  find_num[words[i]] = i

num_words = len(find_word)
num_words

### Environment
Based on https://www.tensorflow.org/agents/tutorials/2_environments_tutorial#creating_your_own_python_environment

In [ ]:
import random

def get_observation_array(game):
  obs = np.zeros(26+26+5)
  for row in range(len(game.board)):
    for col in range(len(game.board[row])):
      char = game.board[row][col].char
      color = game.board[row][col].color
      if char == 'A':
        print("A COLOR " + str(color) + " TO " + str(int(color)))
        print("GAME COLOR " + str(int(game.board[0][1].color)))

      if color == Color.GREY:
        obs[ord(char)-65] = 1
      elif color == Color.YELLOW:
        obs[26+ord(char)-65] = 1
      elif color == Color.GREEN:
        obs[26+26+ord(char)-65] = 1

  return obs

def get_reward(game):
  sum = 0
  for row in game.board:
    for tile in row:
      sum += int(tile.color)

game = WordleGame(find_word[random.randint(0,num_words)])
game.guess("LATER")
print(game.answer)
print(get_observation_array(game)[:26])
print(get_observation_array(game)[26:26+26])
print(get_observation_array(game)[26+26:])

TypeError: ignored

In [ ]:
class WordleEnvironment(py_environment.PyEnvironment):
  def __init__(self):
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int32, minimum=0, maximum=10657, name='action'
    )
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(5*6,), dtype=np.int32, minimum=0, name='observation'
    )
    # self._state = 0
    self.game = WordleGame(find_word(random.randint(num_words)))

  def action_spec(self):
    return self._action_spec
  
  def observation_spec(self):
    return self.observation_spec
  
  def _reset(self):
    self.game = WordleGame(find_word(random.randint(num_words)))
    return ts.restart(np.array([self._state], dtype=np.int32)) # TODO update

  def _step(self, action):
    if self.game.is_over: 
      return self.reset()

    # perform the action
    colors = self.game.guess(find_word[action])

    # return the observation state
    # if the game is now over
    if self.game.is_over:
      return ts.termination(np.)


    


        

In [ ]:
num_iterations = 20000 # @param {type:"integer"}

initial_collect_steps = 100  # @param {type:"integer"}
collect_steps_per_iteration =   1# @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}